In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error, mean_squared_error
from prophet import Prophet
historical_scope=pd.read_csv('../data/historical_scope.csv')

In [2]:
historical_scope

,Isin,Year,EG017,EG018,EG030,IS010,Industry,Sum_EG_25,Sum_EG_50,Sum_EG_75
0,US0079031078,2010,26.628,140.384,286.840,6494.000000,Semiconductors,0.836169,7.411292,33.869677
1,US0326541051,2010,34.933,96.687,14.211,2761.503000,Semiconductors,0.836169,7.411292,33.869677
2,US0382221051,2010,42.172,133.960,36.863,9549.000000,Semiconductors,0.836169,7.411292,33.869677
3,US11135F1012,2010,110.810,156.699,10.771,2093.000000,Semiconductors,0.836169,7.411292,33.869677
4,US29355A1079,2010,0.005,9.199,14.247,61.661000,Semiconductors,0.836169,7.411292,33.869677
...,...,...,...,...,...,...,...,...,...,...
7723,NO0012916131,2023,0.016,0.066,9.068,1089.491000,Software and Tech Services,0.255928,2.110228,21.872761
7724,US00676P1075,2023,8.514,3.846,2.522,388.788000,Software and Tech Services,0.255928,2.110228,21.872761
7725,IT0005433740,2023,1.665,1.107,0.909,91.023833,Software and Tech Services,0.255928,2.110228,21.872761
7726,CNE0000001Y0,2023,7.336,6.103,2.842,2858.122182,Software and Tech Services,0.255928,2.110228,21.872761


In [3]:
historical_scope[historical_scope.isnull().any(axis=1)]

,Isin,Year,EG017,EG018,EG030,IS010,Industry,Sum_EG_25,Sum_EG_50,Sum_EG_75
15,US74736K1016,2010,NaN,NaN,NaN,NaN,Semiconductors,0.836169,7.411292,33.869677
20,CNE100004ZD8,2010,NaN,NaN,NaN,NaN,Semiconductors,0.836169,7.411292,33.869677
21,US00108J1097,2010,NaN,NaN,NaN,NaN,Semiconductors,0.836169,7.411292,33.869677
22,TW0003438008,2010,NaN,NaN,NaN,NaN,Semiconductors,0.836169,7.411292,33.869677
23,CNE100003MM9,2010,NaN,NaN,NaN,NaN,Semiconductors,0.836169,7.411292,33.869677
...,...,...,...,...,...,...,...,...,...,...
7281,MT0000400102,2021,4.473,3.286,NaN,38.679863,Software and Tech Services,0.313765,2.815279,25.429114
7485,MT0000400102,2022,4.708,1.446,NaN,37.516061,Software and Tech Services,0.287454,2.717840,22.757420
7594,US53261M2035,2023,NaN,NaN,NaN,NaN,Software and Tech Services,0.255928,2.110228,21.872761
7689,MT0000400102,2023,2.615,1.666,NaN,39.671991,Software and Tech Services,0.255928,2.110228,21.872761


In [4]:
historical_scope = historical_scope.dropna(subset=['EG017', 'EG018'])

In [5]:
historical_scope['s1s2']=historical_scope['EG017']+historical_scope['EG018']

C:\Users\Veronica\AppData\Local\Temp\ipykernel_49852\3427789118.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  historical_scope['s1s2']=historical_scope['EG017']+historical_scope['EG018']


In [6]:
historical_scope

,Isin,Year,EG017,EG018,EG030,IS010,Industry,Sum_EG_25,Sum_EG_50,Sum_EG_75,s1s2
0,US0079031078,2010,26.628,140.384,286.840,6494.000000,Semiconductors,0.836169,7.411292,33.869677,167.012
1,US0326541051,2010,34.933,96.687,14.211,2761.503000,Semiconductors,0.836169,7.411292,33.869677,131.620
2,US0382221051,2010,42.172,133.960,36.863,9549.000000,Semiconductors,0.836169,7.411292,33.869677,176.132
3,US11135F1012,2010,110.810,156.699,10.771,2093.000000,Semiconductors,0.836169,7.411292,33.869677,267.509
4,US29355A1079,2010,0.005,9.199,14.247,61.661000,Semiconductors,0.836169,7.411292,33.869677,9.204
...,...,...,...,...,...,...,...,...,...,...,...
7723,NO0012916131,2023,0.016,0.066,9.068,1089.491000,Software and Tech Services,0.255928,2.110228,21.872761,0.082
7724,US00676P1075,2023,8.514,3.846,2.522,388.788000,Software and Tech Services,0.255928,2.110228,21.872761,12.360
7725,IT0005433740,2023,1.665,1.107,0.909,91.023833,Software and Tech Services,0.255928,2.110228,21.872761,2.772
7726,CNE0000001Y0,2023,7.336,6.103,2.842,2858.122182,Software and Tech Services,0.255928,2.110228,21.872761,13.439


In [7]:
historical_scope = historical_scope[historical_scope['Year'] != 2023]


In [8]:
historical_scope

,Isin,Year,EG017,EG018,EG030,IS010,Industry,Sum_EG_25,Sum_EG_50,Sum_EG_75,s1s2
0,US0079031078,2010,26.628,140.384,286.840,6494.000000,Semiconductors,0.836169,7.411292,33.869677,167.012
1,US0326541051,2010,34.933,96.687,14.211,2761.503000,Semiconductors,0.836169,7.411292,33.869677,131.620
2,US0382221051,2010,42.172,133.960,36.863,9549.000000,Semiconductors,0.836169,7.411292,33.869677,176.132
3,US11135F1012,2010,110.810,156.699,10.771,2093.000000,Semiconductors,0.836169,7.411292,33.869677,267.509
4,US29355A1079,2010,0.005,9.199,14.247,61.661000,Semiconductors,0.836169,7.411292,33.869677,9.204
...,...,...,...,...,...,...,...,...,...,...,...
7519,NO0012916131,2022,0.020,0.071,11.754,1323.126000,Software and Tech Services,0.287454,2.717840,22.757420,0.091
7520,US00676P1075,2022,9.323,4.445,2.641,438.933000,Software and Tech Services,0.287454,2.717840,22.757420,13.768
7521,IT0005433740,2022,0.185,0.120,0.647,60.662879,Software and Tech Services,0.287454,2.717840,22.757420,0.305
7522,CNE0000001Y0,2022,6.891,5.437,2.850,2542.244067,Software and Tech Services,0.287454,2.717840,22.757420,12.328


In [9]:

def fit_prophet_model_and_forecast(data, isin):
    # Filtrar los datos para el Isin actual
    data_isin = data[data['Isin'] == isin]


    data_isin = data_isin.rename(columns={'Year': 'ds', 's1s2': 'y'}) 
    

    data_isin['ds'] = pd.to_datetime(data_isin['ds'], format='%Y')
    
    if 'ds' not in data_isin or 'y' not in data_isin:
        raise ValueError("El DataFrame filtrado y renombrado no contiene las columnas 'ds' y 'y' necesarias para Prophet.")
    
    model = Prophet()

    model.fit(data_isin[['ds', 'y']])
    
    future = model.make_future_dataframe(periods=27, freq='Y') 
  
    forecast = model.predict(future)
    
    forecast_2024_2050 = forecast[forecast['ds'].dt.year >= 2023]
 
    projected_values = forecast_2024_2050[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
    
    projected_values['Isin'] = isin
    
    return projected_values




In [10]:
all_projections = []

# Iterar sobre cada ISIN único y ajustar el modelo
for isin in historical_scope['Isin'].unique():
    try:
        projection = fit_prophet_model_and_forecast(historical_scope, isin)
        if projection is not None:
            all_projections.append(projection)
    except ValueError as e:
        print(e)  # Imprimir el mensaje de error si hay un problema con las columnas

# Combinar todas las proyecciones en un solo DataFrame
if all_projections:
    all_projections_df = pd.concat(all_projections, ignore_index=True)
    print(all_projections_df)
else:
    print("No se encontraron proyecciones válidas.")

15:06:26 - cmdstanpy - INFO - Chain [1] start processing
15:06:26 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\Veronica\AppData\Local\Temp\ipykernel_49852\1372155976.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  projected_values['Isin'] = isin
15:06:26 - cmdstanpy - INFO - Chain [1] start processing
15:06:26 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\Veronica\AppData\Local\Temp\ipykernel_49852\1372155976.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  projected_values['Is

              ds       yhat  yhat_lower  yhat_upper          Isin
0     2023-12-31 -27.366677  -78.496387   31.088040  US0079031078
1     2024-12-31 -33.040643  -85.433397   22.822856  US0079031078
2     2025-12-31 -52.261941 -109.497226    0.954973  US0079031078
3     2026-12-31 -71.995907 -131.237608  -13.053827  US0079031078
4     2027-12-31 -92.237518 -148.359452  -38.986759  US0079031078
...          ...        ...         ...         ...           ...
14347 2044-12-31  32.163048   32.163046   32.163049  FR0014005DA7
14348 2045-12-31  33.951397   33.951395   33.951399  FR0014005DA7
14349 2046-12-31  35.722661   35.722659   35.722664  FR0014005DA7
14350 2047-12-31  37.474634   37.474632   37.474636  FR0014005DA7
14351 2048-12-31  36.621856   36.621854   36.621859  FR0014005DA7

[14352 rows x 5 columns]


C:\Users\Veronica\AppData\Local\Temp\ipykernel_49852\1372155976.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  projected_values['Isin'] = isin


In [11]:
all_projections_df

,ds,yhat,yhat_lower,yhat_upper,Isin
0,2023-12-31,-27.366677,-78.496387,31.088040,US0079031078
1,2024-12-31,-33.040643,-85.433397,22.822856,US0079031078
2,2025-12-31,-52.261941,-109.497226,0.954973,US0079031078
3,2026-12-31,-71.995907,-131.237608,-13.053827,US0079031078
4,2027-12-31,-92.237518,-148.359452,-38.986759,US0079031078
...,...,...,...,...,...
14347,2044-12-31,32.163048,32.163046,32.163049,FR0014005DA7
14348,2045-12-31,33.951397,33.951395,33.951399,FR0014005DA7
14349,2046-12-31,35.722661,35.722659,35.722664,FR0014005DA7
14350,2047-12-31,37.474634,37.474632,37.474636,FR0014005DA7


In [12]:
projections=all_projections_df.copy()

In [13]:
projections['ds'] = pd.to_datetime(projections['ds']).dt.year



In [14]:
projections = projections.rename(columns={ 'ds': 'Year'}) 
projections

,Year,yhat,yhat_lower,yhat_upper,Isin
0,2023,-27.366677,-78.496387,31.088040,US0079031078
1,2024,-33.040643,-85.433397,22.822856,US0079031078
2,2025,-52.261941,-109.497226,0.954973,US0079031078
3,2026,-71.995907,-131.237608,-13.053827,US0079031078
4,2027,-92.237518,-148.359452,-38.986759,US0079031078
...,...,...,...,...,...
14347,2044,32.163048,32.163046,32.163049,FR0014005DA7
14348,2045,33.951397,33.951395,33.951399,FR0014005DA7
14349,2046,35.722661,35.722659,35.722664,FR0014005DA7
14350,2047,37.474634,37.474632,37.474636,FR0014005DA7


In [15]:
projections = projections[['Isin','Year', 'yhat']]

In [16]:
projections 

,Isin,Year,yhat
0,US0079031078,2023,-27.366677
1,US0079031078,2024,-33.040643
2,US0079031078,2025,-52.261941
3,US0079031078,2026,-71.995907
4,US0079031078,2027,-92.237518
...,...,...,...
14347,FR0014005DA7,2044,32.163048
14348,FR0014005DA7,2045,33.951397
14349,FR0014005DA7,2046,35.722661
14350,FR0014005DA7,2047,37.474634


In [17]:
projections.to_csv('../data/projections.csv', index=False)